In [4]:
!pip install nltk
import nltk
import pandas as pd

In [5]:
# Descargar corpus de texto y modelos entrenados
nltk.download('punkt') # Modelo para tokenizar por oraciones basado en caracteres de puntuación
nltk.download('stopwords') # Corpus con palabras de parada para cada lenguaje
nltk.download('wordnet') # Gran base de datos léxica en inglés con las relaciones entre palabras

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
##################
## Paths para apuntar a la data
###################
path_docs = './data/docs-raw-texts' #path de los documentos
path_queries = './data/queries-raw-texts'

In [8]:
# Pasos de preprocesamiento: para los siguientes puntos,
#  debe preprocesar documentos y consultas mediante tokenización a nivel de palabra,
#  eliminación de palabras vacías, normalización y stemming

import os
from pathlib import Path
import re

class procesamientotexto:

    def __init__(self,path_dir):
        self.path = path_dir
        self.tokens_doc = {} 
        self.word_tok_nltk_es_sw = {}
        self.nltk_lemmaList = {}

    def tokenizacion(self):
        tokenizer = nltk.RegexpTokenizer(r'\w+')
        for doc in os.listdir(self.path):
            path = os.path.join(self.path,doc)
            content_archivo = open(path,encoding='utf8').read()
            texto = re.match('[\w\W]+<raw><!\[CDATA\[(?P<texto>(.|\n|\s|\s)+)\]\]></raw>',content_archivo).groupdict()['texto'].lower()
            # self.tokens_doc[doc] = nltk.word_tokenize(texto,preserve_line=True)
            self.tokens_doc[doc] = tokenizer.tokenize(texto)
            
        return self.tokens_doc
    
    def stopwords(self):
        nltk_stop_words_es = set(nltk.corpus.stopwords.words('english'))
        for name_doc,doc in self.tokens_doc.items():
            self.word_tok_nltk_es_sw[name_doc] = [token for token in doc if token not in nltk_stop_words_es ]
        return self.word_tok_nltk_es_sw
    
    def stemming(self): 
        wordnet_lemmatizer = nltk.stem.WordNetLemmatizer()
        index = 0
        for name_doc,doc in self.word_tok_nltk_es_sw.items():
            index += 1
            self.nltk_lemmaList[name_doc] = {
                'index': int(re.match('wes2015.(d|q)(?P<num>\d+).naf',name_doc).groupdict()['num']),
                'text': [ wordnet_lemmatizer.lemmatize(word) for word in doc ]
            }
        return self.nltk_lemmaList
    
    def dicterminos(self):
        dic = set([])
        for doc in self.nltk_lemmaList.values():
            dic = dic.union(set(doc['text'])) 
        return dic

    
text_process = procesamientotexto(path_docs)
doc_tokens = text_process.tokenizacion()
word_tok_nltk_es_sw = text_process.stopwords()
nltk_lemmaList = text_process.stemming()  # los textos lemmatizados
dicterminos = text_process.dicterminos()  #vocabulario

print(nltk_lemmaList['wes2015.d001.naf'])
print(dicterminos)


{'index': 1, 'text': ['william', 'beaumont', 'human', 'digestion', 'william', 'beaumont', 'physiology', 'digestion', 'image', 'source', 'november', '21', '1785', 'u', 'american', 'surgeon', 'william', 'beaumont', 'born', 'became', 'best', 'known', 'father', 'gastric', 'physiology', 'following', 'research', 'human', 'digestion', 'william', 'beaumont', 'born', 'lebanon', 'connecticut', 'became', 'physician', 'served', 'surgeon', 'mate', 'army', 'war', '1812', 'opened', 'private', 'practice', 'plattsburgh', 'new', 'york', 'rejoined', 'army', 'surgeon', '1819', 'beaumont', 'stationed', 'fort', 'mackinac', 'mackinac', 'island', 'michigan', 'early', '1820s', 'existed', 'protect', 'interest', 'american', 'fur', 'company', 'fort', 'became', 'refuge', 'wounded', '19', 'year', 'old', 'french', 'canadian', 'fur', 'trader', 'named', 'alexis', 'st', 'martin', 'shotgun', 'went', 'accident', 'american', 'fur', 'company', 'store', 'close', 'range', 'june', '6th', '1822', 'st', 'martin', 'wound', 'quit

In [12]:
# [10p] Cree su propia implementación del índice invertido usando 
# los 331 documentos en el conjunto de datos.

def indiceinvertido(doc_lemalist: dict,terminos:dict):
    indiceinvertido = {}
    for termino in terminos: 
         indiceinvertido[termino] = {'IDdocs':[],'len':0}
  
    for documento in doc_lemalist.values():   ## no me gusta el doble for
        set_texto = set(documento['text'])
        for termino in set_texto: 
            indiceinvertido[termino]['IDdocs'].append(documento['index'])
            indiceinvertido[termino]['len'] +=1 
    return indiceinvertido


list_indiceinvertido = indiceinvertido(nltk_lemmaList,dicterminos)
print(list_indiceinvertido)

{'criticize': {'IDdocs': [131], 'len': 1}, 'realm': {'IDdocs': [87, 139, 249], 'len': 3}, 'fence': {'IDdocs': [134], 'len': 1}, 'outspoken': {'IDdocs': [137, 203], 'len': 2}, 'startupbootcamp': {'IDdocs': [195], 'len': 1}, 'nicholas': {'IDdocs': [12, 35, 226], 'len': 3}, 'boris': {'IDdocs': [130], 'len': 1}, 'usedom': {'IDdocs': [103], 'len': 1}, 'polemical': {'IDdocs': [241], 'len': 1}, 'asteroid': {'IDdocs': [38, 42, 61], 'len': 3}, 'anywhere': {'IDdocs': [129, 161, 199], 'len': 3}, 'szilárd': {'IDdocs': [5, 251], 'len': 2}, '247': {'IDdocs': [331], 'len': 1}, 'preliminary': {'IDdocs': [107, 141, 250, 323], 'len': 4}, 'rye': {'IDdocs': [187], 'len': 1}, 'four': {'IDdocs': [6, 9, 10, 18, 29, 33, 34, 36, 42, 49, 55, 63, 64, 65, 69, 75, 76, 84, 92, 98, 99, 106, 108, 109, 115, 116, 119, 128, 133, 134, 135, 138, 142, 145, 146, 148, 154, 158, 165, 176, 178, 181, 183, 184, 189, 190, 191, 192, 199, 205, 209, 212, 214, 227, 234, 235, 243, 244, 249, 258, 260, 262, 263, 271, 272, 277, 281, 294,

In [16]:
# [10p] Cree una función que lea el índice invertido y calcule consultas booleanas mediante el algoritmo de mezcla.
# El algoritmo de mezcla debe ser capaz de calcular: AND, y NOT.

def And(indiceInvertido: dict,termino1: str,termino2: str):
    intersect_id = []
     
    indices_doc_t1 = indiceInvertido[termino1]['IDdocs']
    indices_doc_t2 = indiceInvertido[termino2]['IDdocs']
 
    i,j = 0,0 
    while  i<len(indices_doc_t1) and j<len(indices_doc_t2):
        if indices_doc_t1[i] == indices_doc_t2[j]:
            intersect_id.append(indices_doc_t1[i])
            i,j = i+1,j+1
        elif indices_doc_t1[i]>indices_doc_t2[j]: 
            j += 1
        else: 
            i += 1
    return intersect_id

# set1 = set(list_indiceinvertido['despite']['IDdocs'])
# set2 = set(list_indiceinvertido['plan']['IDdocs'])
# print(set1.intersection(set2))
print(And(list_indiceinvertido,'despite','plan'))

def Not(indiceInvertido: dict,termino:str = None, lstTerminos: str = None): 
    
    if termino is not None: 
        setTerminos = set(indiceInvertido[termino])
    else: 
        setTerminos = lstTerminos
    setUniversal = set(range(1,332))
    return list(setUniversal - setTerminos)


print(Not(list_indiceinvertido,'despite'))

{26, 3, 277, 134}
[3, 26, 134, 277]


In [32]:
# [5p] Para cada una de las 35 consultas en el conjunto de datos, 
# recupere los documentos utilizando consultas binarias
#  AND (i.e. termino_1 AND termino_2 AND termino_3…). 
# Escriba un archivo (BSII-AND-queries_results) 
# con los resultados siguiendo el mismo formato que "relevance-judgments":
# q01 dXX,dYY,dZZ…
# Nota: pueden resultar archivos vacíos.
# path_queries

proc_querys = procesamientotexto(path_queries)
tokens_querys = proc_querys.tokenizacion()
tokens_querys_sw = proc_querys.stopwords()
terminos_querys = proc_querys.stemming()
print(terminos_querys)

def queries(list_indiceinvertido,terminos_querys):

    data = {'query':[],'results':[]}
    def And2(indiceinvertido:dict,query: list):
        
        IdDocs_list = []
        for termino in query:
            if termino in indiceinvertido: 
                IdDocs_list.append(indiceinvertido[termino]['IDdocs'])
            else: 
                return []
            
        and_set = set(IdDocs_list[0])
        for iddocs in IdDocs_list:
            and_set=and_set.intersection(iddocs)
        
        return and_set
    
    for docquery in terminos_querys.values():
        indice = docquery['index']
        texto = docquery['text']
        results = list(And2(list_indiceinvertido,texto))
        data['query'].append(f'q{indice:02d}')
        
        result = ','.join(
            list(map(lambda x: f'd{x:03d}',results))
            ) if len(results) != 0 else ''
        
        data['results'].append(result)

    df = pd.DataFrame(data)

    return df 


df_q = queries(list_indiceinvertido,terminos_querys)
df_q.head()
df_q.to_csv('BSII-AND-queries_results.tsv',sep='\t',header=False,index=False)


{'wes2015.q01.naf': {'index': 1, 'text': ['fabrication', 'music', 'instrument']}, 'wes2015.q02.naf': {'index': 2, 'text': ['famous', 'german', 'poetry']}, 'wes2015.q03.naf': {'index': 3, 'text': ['romanticism']}, 'wes2015.q04.naf': {'index': 4, 'text': ['university', 'edinburgh', 'research']}, 'wes2015.q06.naf': {'index': 6, 'text': ['bridge', 'construction']}, 'wes2015.q07.naf': {'index': 7, 'text': ['walk', 'fame', 'star']}, 'wes2015.q08.naf': {'index': 8, 'text': ['scientist', 'worked', 'atomic', 'bomb']}, 'wes2015.q09.naf': {'index': 9, 'text': ['invention', 'internet']}, 'wes2015.q10.naf': {'index': 10, 'text': ['early', 'telecommunication', 'method']}, 'wes2015.q12.naf': {'index': 12, 'text': ['explored', 'south', 'pole']}, 'wes2015.q13.naf': {'index': 13, 'text': ['famous', 'member', 'royal', 'navy']}, 'wes2015.q14.naf': {'index': 14, 'text': ['nobel', 'prize', 'winning', 'invention']}, 'wes2015.q16.naf': {'index': 16, 'text': ['south', 'america']}, 'wes2015.q17.naf': {'index': 

In [23]:
def And2(indiceinvertido:dict,query: list):
    len_terminos_q = len(query)
    IdDocs_list = []
    for termino in query: 
        IdDocs_list.append(indiceinvertido[termino]['IDdocs'])
    and_set = set(IdDocs_list[0])
    for iddocs in IdDocs_list:
        and_set=and_set.intersection(iddocs)
    
    return and_set

And2(list_indiceinvertido, ['famous', 'german', 'poetry'])

{291, 293}